## Accessing DM-EFD data


In this notebook we demonstrate the LSST EFD client.
The client docs can be found [here](https://efd-client.lsst.io).

This is complementaty to the [Chronograf](https://lsst-chronograf-int-efd.ncsa.illinois.edu/) interface which we use for time-series visualization.

In [ ]:
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

We'll access the EFD instance deployed at the LDF.
You need to be on-site or connected to a VPN that can see the LDF network.
See see the [docs](https://nb.lsst.io) for help logging in. 

In [ ]:
client = EfdClient('ldf_stable_efd')

## Listing topics
Topics are mapped to [InfluxDB measurements](https://docs.influxdata.com/influxdb/v1.7/concepts/key_concepts/#measurement). The following query simply lists all topics in the InfluxDB `efd` database:

In [ ]:
topics = await client.get_topics()
topics[:15]

## Querying topic data

In [ ]:
t2 = Time('2020-03-16T12:00:00', scale='tai')
t1 = t2 - TimeDelta(15*24*3600, format='sec', scale='tai')

In [ ]:
df = await client.select_time_series('lsst.sal.ATCamera.wreb', '*', t1, t2)
df.head()

At this point you can use [Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/index.html) to analyze the data. First plot all CCD temperature data at full resolution.

In [ ]:
df.plot(y='ccdTemp0')

Now average data over 15 minute intervals and plot.

In [ ]:
ccdTemp01h = df['ccdTemp0'].resample('15min').mean()

In [ ]:
ccdTemp01h.plot()

Finally, we can look at correlations between values in the database.

In [ ]:
df.plot.scatter(x='temp1', y='temp2')

## Querying annotations

In Chronograf, you can annotate the time-series to mark interesting events or features in the data. These annotations are saved in the `chronograf` database and can also be queried.

In [ ]:
df = await client.influx_client.query('SELECT * FROM "chronograf"."autogen"."annotations"')
df['text']